Here we start with analysis of health data using several different measures described in 
https://jakobrunge.github.io/tigramite/#tigramite.pcmci.PCMCI and these https://towardsdatascience.com/inferring-causality-in-time-series-data-b8b75fe52c46#586a 


In [2]:
#loading data in format of arrays var(t) - variable in time 

######################
# data is array with
# np.shape(data) = (T, n)
# where T is time stamps 
# n is number of variables
######################

In [ ]:
# load data from variable1 and variable2 ...
data12 = np.column_stack((var1, var2))

data = np.column_stack((data12, var3))
print('shape of data with symptoms', data.shape)
print('loaded data with 3 variables')


# Method application

    Plotting of data
    Nonlinear conditional independence tests
    Symbolic time series


In [ ]:
# Imports
import numpy
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
import sklearn

import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb
from tigramite.models import LinearMediation, Prediction


In [ ]:

#data, true_parents_neighbors = pp.var_process(links_coeffs, T=T)
T, N = data.shape

# Initialize dataframe object from data
dataframe = pp.DataFrame(data)
dataframe_norm = pp.DataFrame(data_normal)

# Specify time axis and variable names
datatime = numpy.arange(len(data))
var_names = [r'$X^0$', r'$X^1$', r'$X^2$', r'$X^3$']

# PCMCI method applied to data

PCMCI estimates causal links by a two-step procedure:

   1. Condition-selection: For each variable j, estimate a superset of parents $\tilde{\mathcal{P}}(X^j_t)$ with the iterative PC1 algorithm , implemented as run_pc_stable.

   2. Momentary conditional independence (MCI)   $  X^i_{t-\tau} ~\perp~ X^j_{t} ~|~ \tilde{\mathcal{P}}(X^j_t), \tilde{\mathcal{P}}(X^i_{t-{\tau}})$

here implemented as run_mci. The condition-selection step reduces the dimensionality and avoids conditioning on irrelevant variables.

Then we initialize the ``PCMCI`` method with  ``dataframe``, ``cond_ind_test``, and (optionally) ``var_names``.

In [ ]:
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb
from tigramite.models import LinearMediation, Prediction

# setting up the parameters for the PCMCI method

parcorr = ParCorr(significance='analytic')

pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=parcorr,
    verbosity=1)

print('pcmci variable outcome type', type(pcmci))

'''
deleted the element in pcmci method:   var_names=var_names not to get errors
'''

In [ ]:
# apply pcmci method to other data with other variables

parcorr = ParCorr(significance='analytic')

pcmci_norm = PCMCI(
    dataframe=dataframe_norm, 
    cond_ind_test=parcorr,
    verbosity=1)


# Correlations¶

Before running the causal algorithm, it's a good idea to plot the lagged unconditional dependencies, e.g., the lagged correlations. This can help to identify which maximal time lag tau_max to choose in the causal algorithm.


In [ ]:

correlations = pcmci.get_lagged_dependencies(tau_max=20)
lag_func_matrix = tp.plot_lagfuncs(val_matrix=correlations, setup_args={'var_names':var_names, 
                                    'x_base':5, 'y_base':.5})